This notebook set's up an FEM simulation by sampling from the analytical approximation of the surface patch with some $\theta$. Then we translate this into a simnibs input matrix and run a simulation and return the results

In [1]:
import sys

In [2]:
#SCC package environment
sys.path.insert(0,'/KIMEL/tigrlab/projects/jjeyachandra/gmsh-sdk/lib/')
sys.path.insert(0,'/home/jjeyachandra/simnibs_2.1.2/miniconda2/envs/simnibs_env/lib/python2.7/site-packages')
sys.path.insert(0,'/home/jjeyachandra/simnibs_2.1.2/Python_modules/src')

In [3]:
import numpy as np
from simnibs import sim_struct, run_simulation

#For importing geometry library
import io, os, sys, types
from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell

#Timing
from timeit import default_timer as timer

In [1]:
#https://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/Importing%20Notebooks.html
#This is for loading functions for another jupyter notebook
class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path

    def load_module(self, fullname):
        """import a notebook as a module"""
        
        
        path = os.path.join(self.path,fullname) + '.ipynb'

        print ("importing Jupyter notebook from %s" % path)

        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)


        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod

        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__

        try:
            for cell in nb.cells:
                if cell.cell_type == 'code':
                    # transform the input to executable Python
                    code = self.shell.input_transformer_manager.transform_cell(cell.source)
                    # run the code in themodule
                    exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod

In [5]:
#Load up the geolibrary as a module
glib_path = '/projects/jjeyachandra/rtms_optimize/notebooks/python2/'
nbload = NotebookLoader(glib_path)
geolib = nbload.load_module('geolib')

importing Jupyter notebook from /projects/jjeyachandra/rtms_optimize/notebooks/python2/geolib.ipynb


In [6]:
#Load up quadratic surface constants, inverse rotation matrix, bounds and normal vector
#This is for the Bayesian optimization component
C = np.fromfile('../../output/quadratic_vec')
iR = np.fromfile('../../output/inverse_rot').reshape((3,3))
b  = np.fromfile('../../output/param_bounds').reshape((3,2))

### Get raw input coordinates that need to be transformed

In [7]:
#Sample a coordinate within the bounds and form input array
x = np.random.uniform(b[0,0], b[0,1])
y = np.random.uniform(b[1,0], b[1,1])

#Sample rotation
r = np.random.uniform(0,180)
print(x,y,r)

(16.513799154644985, 6.895013488831868, 72.53564436856989)


In [12]:
#Compute z value then map to mesh surface
preaff_loc = geolib.map_param_2_surf(x,y,C)
loc = np.matmul(iR,preaff_loc)

#Compute orientation vector in basis of surface normal then rotate into FEM mesh space
preaff_rot, preaff_n = geolib.map_rot_2_surf(x,y,r,C)
rot = np.matmul(iR,preaff_rot)
n = np.matmul(iR,preaff_n)

In [20]:
#Start a session
S = sim_struct.SESSION()
S.fnamehead="../../data/simnibs_output/sub-CMH090.msh"
S.pathfem="../../data/simnibs_output/test_simulation"

In [21]:
#Define the TMS simulation
tms = S.add_tmslist()
tms.fnamecoil='/projects/jjeyachandra/simnibs/ccd-files/Magstim_70mm_Fig8.nii.gz'

In [22]:
#Define position
pos = tms.add_position()
pos.matsimnibs = geolib.define_coil_orientation(loc,rot,n)

In [23]:
start = timer()
run_simulation(S)
stop = timer()
print(stop - start)

[ simnibs ]INFO: Could not find the dti tensor file in the standard location: /mnt/tigrlab/projects/jjeyachandra/rtms_optimize/data/simnibs_output/d2c_sub-CMH090/dti_results_T1space/DTI_conf_tensor.nii.gz
[ simnibs ]INFO: Head Mesh: /mnt/tigrlab/projects/jjeyachandra/rtms_optimize/data/simnibs_output/sub-CMH090.msh
[ simnibs ]INFO: Subject Path: /mnt/tigrlab/projects/jjeyachandra/rtms_optimize/data/simnibs_output/m2m_sub-CMH090
[ simnibs ]INFO: Simulation Folder: /mnt/tigrlab/projects/jjeyachandra/rtms_optimize/data/simnibs_output/test_simulation
[ simnibs ]INFO: Running simulations in the directory: /mnt/tigrlab/projects/jjeyachandra/rtms_optimize/data/simnibs_output/test_simulation
[ simnibs ]INFO: Running Poslist Number: 1
[ simnibs ]INFO: Began to run TMS simulations
[ simnibs ]INFO: Using coil: /projects/jjeyachandra/simnibs/ccd-files/Magstim_70mm_Fig8.nii.gz
[ simnibs ]INFO: Using isotropic conductivities


0


[ simnibs ]INFO: Running TMS Position 0
[ simnibs ]INFO: Coil Position Matrix: [[ -0.168   -0.7362   0.6556 -56.6009]
 [  0.6209  -0.5956  -0.5096  54.8834]
 [ -0.7656  -0.3214  -0.5572  42.0694]
 [  0.       0.       0.       1.    ]]
dIdt: 1000000.0

[ simnibs ]INFO: Executing: 
/home/jjeyachandra/simnibs_2.1.2/bin/getdp /mnt/tigrlab/projects/jjeyachandra/rtms_optimize/data/simnibs_output/test_simulation/sub-CMH090_TMS_1-0001_Magstim_70mm_Fig8_nii.pro -solve QS -msh /mnt/tigrlab/projects/jjeyachandra/rtms_optimize/data/simnibs_output/test_simulation/tmpROVqjb.msh -ksp_type cg -ksp_rtol 1e-10 -pc_type icc -pc_factor_levels 2 -v2 -bin
[ simnibs ]INFO: Info    : Running '/home/jjeyachandra/simnibs_2.1.2/bin/getdp /mnt/tigrlab/projects/jjeyachandra/rtms_optimize/data/simnibs_output/test_simulation/sub-CMH090_TMS_1-0001_Magstim_70mm_Fig8_nii.pro -solve QS -msh /mnt/tigrlab/projects/jjeyachandra/rtms_optimize/data/simnibs_output/test_simulation/tmpROVqjb.msh -ksp_type cg -ksp_rtol 1e-10 -p

[ simnibs ]INFO: Info    :  46 KSP Residual norm 1.041465454416e-03
[ simnibs ]INFO: Info    :  47 KSP Residual norm 8.003410277087e-04
[ simnibs ]INFO: Info    :  48 KSP Residual norm 6.461267706856e-04
[ simnibs ]INFO: Info    :  49 KSP Residual norm 5.007001068500e-04
[ simnibs ]INFO: Info    :  50 KSP Residual norm 3.803303581025e-04
[ simnibs ]INFO: Info    :  51 KSP Residual norm 2.991108950449e-04
[ simnibs ]INFO: Info    :  52 KSP Residual norm 2.427475608809e-04
[ simnibs ]INFO: Info    :  53 KSP Residual norm 1.940761013340e-04
[ simnibs ]INFO: Info    :  54 KSP Residual norm 1.601560706432e-04
[ simnibs ]INFO: Info    :  55 KSP Residual norm 1.312494888834e-04
[ simnibs ]INFO: Info    :  56 KSP Residual norm 1.107458354683e-04
[ simnibs ]INFO: Info    :  57 KSP Residual norm 9.152548047183e-05
[ simnibs ]INFO: Info    :  58 KSP Residual norm 7.706190565202e-05
[ simnibs ]INFO: Info    :  59 KSP Residual norm 6.758850226648e-05
[ simnibs ]INFO: Info    :  60 KSP Residual norm

371.256185055
